In [62]:
from pool import ConcLiquidityPool, AMMPoolHistory

In [63]:
# Define token0=USDC and token1=WETH
token0 = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'
token1 = '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'

In [64]:
# Create an instance of AMMPoolHistory
feedlot = AMMPoolHistory(1000000000, 666666)

In [65]:
# instantiate Streamer object. 
# Note - unlike the cow queries, univ3 does not require multiple streamer instantations because the swaps field path is reset each iteration. 
# If the Cow queries were updated to use the same method, we could use the same streamer object for all queries.
univ3_ds = Streamer('https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum')

# get a query field path from the query dictionary which is automatically populated in the Streamer object
swaps_fp = univ3_ds.queryDict.get('swaps')

# define query path and parameters 
uni_swaps_qp = uni_swaps_fp(
        first=100,
        orderBy='timestamp',
        orderDirection='desc',
        where = {'pool': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'} 
        )

# run query
uni_swaps_df = univ3_ds.runQuery(uni_swaps_qp)

FIELD - swaps


In [66]:
uni_swaps_df.shape

(100, 20)

In [67]:
uni_swaps_df.dtypes

swaps_id               object
swaps_hash             object
swaps_nonce             int64
swaps_logIndex          int64
swaps_gasLimit          int64
swaps_gasUsed          object
swaps_gasPrice          int64
swaps_protocol_id      object
swaps_account_id       object
swaps_pool_id          object
swaps_blockNumber       int64
swaps_timestamp         int64
swaps_tick              int64
swaps_tokenIn_id       object
swaps_amountIn         object
swaps_amountInUSD     float64
swaps_tokenOut_id      object
swaps_amountOut        object
swaps_amountOutUSD    float64
endpoint               object
dtype: object

In [68]:
uni_swaps_df.head(5)

,swaps_id,swaps_hash,swaps_nonce,swaps_logIndex,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_protocol_id,swaps_account_id,swaps_pool_id,swaps_blockNumber,swaps_timestamp,swaps_tick,swaps_tokenIn_id,swaps_amountIn,swaps_amountInUSD,swaps_tokenOut_id,swaps_amountOut,swaps_amountOutUSD,endpoint
0,0x818de833723e06792bde58a67b6a141363ae6cbc487b...,0x818de833723e06792bde58a67b6a141363ae6cbc487b...,342,224,405192,None,32295986143,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xdfab649e190815705bae69e6db098a9778f2546b,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,16793650,1678401371,203589,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,179600000000000000,258.799250,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,258022627,258.022627,uniswap-v3-ethereum
1,0xb2e89bf93a1bfa9943c0900bbce4f240f9ebc710cc08...,0xb2e89bf93a1bfa9943c0900bbce4f240f9ebc710cc08...,51,166,226974,None,34358586607,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x875a24c5fcf88b7e9459c8808df61fe6ce36f7d8,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,16793596,1678400723,203589,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,867942503293009175,1250.694680,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,1246937335,1246.937335,uniswap-v3-ethereum
2,0x234693646344db7b9bbef7f8631ff3899a9df04959d0...,0x234693646344db7b9bbef7f8631ff3899a9df04959d0...,1360,115,442851,None,36319399091,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x71195ac7a1daa8e66d7b38c16860f69b7a20e797,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,16793591,1678400663,203589,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2000000000,2000.000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,1383782193831811975,1993.986546,uniswap-v3-ethereum
3,0x8ae8a6798da9064e4abb5748108cf1e7da59e0a76ba4...,0x8ae8a6798da9064e4abb5748108cf1e7da59e0a76ba4...,69,414,235834,None,36996584759,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xa6506c76f771f35f28f860b0d0cfabaa0182f074,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,16793577,1678400495,203589,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,278425981559450372,401.294344,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,400000000,400.000000,uniswap-v3-ethereum
4,0xcd693ecf4c1ccc4c8f37b46dab6e326e591284022cb6...,0xcd693ecf4c1ccc4c8f37b46dab6e326e591284022cb6...,304,258,246681,None,38879904045,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x354304234329a8d2425965c647e701a72d3438e5,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,16793575,1678400471,203589,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,22967547751179067432,33103.042160,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,33000000000,33000.000000,uniswap-v3-ethereum


In [69]:
# Iterate through the rows of the dataframe
for index, row in uni_swaps_df.iterrows():
    # Determine if the swap is a buy or a sell
    token_in = row['swaps_tokenIn_id']
    token_out = row['swaps_tokenOut_id']
    is_buy = token_in == token0 and token_out == token1
    
    # Extract the amountIn, amountOut, and timestamp
    if is_buy:
        amt0 = row['swaps_amountIn']
        amt1 = row['swaps_amountOut']
    else:
        amt0 = row['swaps_amountOut']
        amt1 = row['swaps_amountIn']
    ts = row['swaps_timestamp']
    
    print('amt0', amt0)
    print('amt1', amt1)
    print('ts', ts)
    
    # Run the swap function on the AMM pool
    pool.swap(amt0, amt1, ts)

amt0 258022627
amt1 179600000000000000
ts 1678401371
amt0 1246937335
amt1 867942503293009175
ts 1678400723
amt0 2000000000
amt1 1383782193831811975
ts 1678400663
amt0 400000000
amt1 278425981559450372
ts 1678400495
amt0 33000000000
amt1 22967547751179067432
ts 1678400471
amt0 215545708
amt1 150000000000000000
ts 1678400423
amt0 6246085223
amt1 4346606151980000005
ts 1678400387
amt0 574814386
amt1 400000000000000000
ts 1678400363
amt0 191356839794
amt1 133074390677642475941
ts 1678400351
amt0 123159376733
amt1 85557177230522164040
ts 1678400147
amt0 282974161522
amt1 196309063269300011008
ts 1678400135
amt0 449813902732
amt1 310355541162599124726
ts 1678400075
amt0 65050722
amt1 44950687752527101
ts 1678400003
amt0 6370000000
amt1 4401828024917012540
ts 1678399991
amt0 223890943917
amt1 154834356506680746084
ts 1678399979
amt0 171237374667
amt1 118581540866837481738
ts 1678399967
amt0 2999613889
amt1 2076005482150637634
ts 1678399967
amt0 40000000
amt1 27715937375235038
ts 1678399955
am

In [70]:
import pandas as pd
df = pd.DataFrame(feedlot.history)

In [71]:
df.shape

(1, 3)